In [64]:
using FLOYao
using Yao
using SkewLinearAlgebra
using Tullio
using FiniteDifferences
using Symbolics
using SymbolicUtils

In [2]:
using Symbolics
# using SymPy

In [3]:
using LinearAlgebra
⊗ = kron

kron (generic function with 72 methods)

In [4]:
using Revise

In [5]:
function matdisplay(mat, digits=2)
    display(round.(mat, digits=digits))
end

matdisplay (generic function with 2 methods)

In [270]:
reg1 = FLOYao.rand_state(3)
reg2 = FLOYao.zero_state(3)

MajoranaReg{Float64} with 3 qubits:
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0

In [271]:
fidelity(reg1, reg2)

0.7540882871860838

In [223]:
function W_matrix(R::Matrix{T}) where {T}
    n = size(R, 1) ÷ 2
    W = zeros(Complex{T}, 2n, 2n)
    for i in 1:2:2n
        W[:,i] .= R[:,i] .- 1im * R[:,i+1]
        W[:,i+1] .= R[:,i] .+ 1im * R[:,i+1]
    end
    return W
end

W_matrix (generic function with 1 method)

In [224]:
function W_matrix_tullio(R::Matrix{T}) where {T}
    n = size(R, 1) ÷ 2
    R = reshape(R, 2, n, 2, n)
    W = zeros(Complex{T}, 2, n, 2, n)
    @tullio W[μ,m,α,i] = R[μ,m,1,i] + 1im*(-1)^α * R[μ,m,2,i]
    return reshape(W, 2n, 2n)
end

W_matrix_tullio (generic function with 1 method)

In [225]:
matdisplay(W_matrix(reg1.state))

4×4 Matrix{ComplexF64}:
  0.6+0.53im   0.6-0.53im  -0.14-0.58im  -0.14+0.58im
  0.2+0.4im    0.2-0.4im    0.81+0.38im   0.81-0.38im
 0.72-0.67im  0.72+0.67im    0.1+0.12im    0.1-0.12im
 0.26+0.33im  0.26-0.33im  -0.56+0.71im  -0.56-0.71im

In [226]:
matdisplay(W_matrix_tullio(reg1.state))

4×4 Matrix{ComplexF64}:
  0.6+0.53im   0.6-0.53im  -0.14-0.58im  -0.14+0.58im
  0.2+0.4im    0.2-0.4im    0.81+0.38im   0.81-0.38im
 0.72-0.67im  0.72+0.67im    0.1+0.12im    0.1-0.12im
 0.26+0.33im  0.26-0.33im  -0.56+0.71im  -0.56-0.71im

In [227]:
function fidelity_observable_matrix(reg::MajoranaReg{T}) where {T}
    n = nqubits(reg)
    R = reg.state
    W = W_matrix(R)
    # matdisplay(W)

    # Pretty sure this could be sped up with some explicit for loops
    # A = transpose(W) * (I(n) ⊗ [1 -1im; 1im 1]) * W / 4
    A_sym = transpose(W) * W / 4
    # matdisplay(A_sym)
    A_asym = transpose(W) * (I(n) ⊗ [0 -1im; 1im 0]) * W / 4
    # matdisplay(A_asym)
    
    A = A_sym + A_asym

    # Take the _correct_ anti-symmetric part of it!
    for i in 1:2n
        for j in i+1:2n
            A[j,i] = -A[i,j]
        end
    end
    A[diagind(A)] .= 0
    return A
end

fidelity_observable_matrix (generic function with 2 methods)

In [228]:
function fidelity_observable_matrix(R)
    n = size(R, 1) ÷ 2
    W = W_matrix(R)
    # matdisplay(W)

    # Pretty sure this could be sped up with some explicit for loops
    # A = transpose(W) * (I(n) ⊗ [1 -1im; 1im 1]) * W / 4
    A_sym = transpose(W) * W / 4
    # matdisplay(A_sym)
    A_asym = transpose(W) * (I(n) ⊗ [0 -1im; 1im 0]) * W / 4
    # matdisplay(A_asym)
    
    A = A_sym + A_asym

    # Take the _correct_ anti-symmetric part of it!
    for i in 1:2n
        for j in i+1:2n
            A[j,i] = -A[i,j]
        end
    end
    A[diagind(A)] .= 0
    return A
end

fidelity_observable_matrix (generic function with 2 methods)

In [229]:
function fidelity_observable_matrix_tullio(reg::MajoranaReg{T}) where {T}
    n = nqubits(reg)
    g = [0 -1im; 1im 0]

    W = reshape(W_matrix_tullio(reg.state), 2, n, 2, n)
    # matdisplay(reshape(W, 2n, 2n))

    @tullio A_sym[α,i,β,j] := W[μ,m,α,i] * W[μ,m,β,j] / 4
    # matdisplay(reshape(A_sym, 2n, 2n))
    @tullio A_asym[α,i,β,j] := W[μ,m,α,i] * g[μ, ν] * W[ν,m,β,j] / 4
    # matdisplay(reshape(A_asym, 2n, 2n))
    @tullio A[α,i,β,j] := A_sym[α,i,β,j] + A_asym[α,i,β,j]
    
    A = reshape(A, 2n, 2n)
    for i in 1:2n
        for j in i+1:2n
            A[j,i] = -A[i,j]
        end
    end
    A[diagind(A)] .= 0
    return A
end

fidelity_observable_matrix_tullio (generic function with 1 method)

In [230]:
A = fidelity_observable_matrix(reg1)
# W = simplify.(W; expand=true)
# A = simplify.(A; expand=true)
matdisplay(A)

4×4 Matrix{ComplexF64}:
   0.0+0.0im   0.22-0.0im    0.41-0.04im   -0.0+0.0im
 -0.22+0.0im    0.0+0.0im     0.0+0.0im   -0.41-0.04im
 -0.41+0.04im  -0.0-0.0im     0.0+0.0im    0.22-0.0im
   0.0-0.0im   0.41+0.04im  -0.22+0.0im     0.0+0.0im

In [231]:
A = fidelity_observable_matrix_tullio(reg1)
matdisplay(A)

4×4 Matrix{ComplexF64}:
   0.0+0.0im   0.22+0.0im    0.41-0.04im    0.0+0.0im
 -0.22-0.0im    0.0+0.0im     0.0+0.0im   -0.41-0.04im
 -0.41+0.04im  -0.0-0.0im     0.0+0.0im    0.22+0.0im
  -0.0-0.0im   0.41+0.04im  -0.22-0.0im     0.0+0.0im

In [232]:
function B_matrix(A_inv)
    n = size(A_inv, 1) ÷ 2
    B = zero(A_inv)
    for i in 1:2:2n
        B[:,i] .= A_inv[:,i] .+ A_inv[:,i+1]
        B[:,i+1] .= -1im .* A_inv[:,i] .+ 1im .* A_inv[:,i+1]
    end
    return B
end

B_matrix (generic function with 1 method)

In [233]:
function B_matrix_tullio(A_inv)
    n = size(A_inv, 1) ÷ 2
    A_inv = reshape(A_inv, 2, n, 2, n)
    δ = Matrix(I(2))
    @tullio B[β,j,γ,g] := A_inv[β,j,1,g] * (δ[γ,1] - 1im * δ[γ,2])  + A_inv[β,j,2,g] * (δ[γ,1] + 1im * δ[γ,2])
    return reshape(B, 2n, 2n)
end

B_matrix_tullio (generic function with 1 method)

In [234]:
A = fidelity_observable_matrix(reg1)
det(A)

0.04883173414591691 + 1.0408340855860843e-17im

In [235]:
A_inv = inv(A)
matdisplay(A_inv)

4×4 Matrix{ComplexF64}:
 -0.0-0.0im    -1.0+0.0im   -1.87-0.19im  -0.0-0.0im
  1.0-0.0im     0.0-0.0im     0.0-0.0im   1.87-0.19im
 1.87+0.19im   -0.0+0.0im     0.0-0.0im   -1.0+0.0im
  0.0+0.0im   -1.87+0.19im    1.0-0.0im    0.0+0.0im

In [236]:
B = B_matrix(A_inv)
matdisplay(B)

4×4 Matrix{ComplexF64}:
  -1.0+0.0im    -0.0-1.0im   -1.87-0.19im  -0.19+1.87im
   1.0-0.0im    -0.0-1.0im    1.87-0.19im   0.19+1.87im
  1.87+0.19im   0.19-1.87im   -1.0+0.0im    -0.0-1.0im
 -1.87+0.19im  -0.19-1.87im    1.0-0.0im    -0.0-1.0im

In [237]:
B = B_matrix_tullio(A_inv)
matdisplay(B)

4×4 Matrix{ComplexF64}:
  -1.0+0.0im    -0.0-1.0im   -1.87-0.19im  -0.19+1.87im
   1.0-0.0im    -0.0-1.0im    1.87-0.19im   0.19+1.87im
  1.87+0.19im   0.19-1.87im   -1.0+0.0im    -0.0-1.0im
 -1.87+0.19im  -0.19-1.87im    1.0-0.0im    -0.0-1.0im

In [238]:
function C_matrix(W)
    n = size(W, 1) ÷ 2
    C = zero(W)
    for i in 1:2:2n
        C[:,i] .= -W[i+1,:] 
        C[:,i+1] .= W[i,:] 
    end
    return C
end

C_matrix (generic function with 1 method)

In [239]:
function C_matrix_tullio(W)
    n = size(W, 1) ÷ 2
    W = reshape(W, 2, n, 2, n)
    g = [0 -1; 1 0]
    ii = Matrix(I(n))
    @tullio C[β,j,δ,d] := W[μ,m,β,j] * ii[d,m] * g[δ,μ]
    return reshape(C, 2n, 2n)
end

C_matrix_tullio (generic function with 1 method)

In [240]:
W = W_matrix(reg1.state)
matdisplay(W)

4×4 Matrix{ComplexF64}:
  0.6+0.53im   0.6-0.53im  -0.14-0.58im  -0.14+0.58im
  0.2+0.4im    0.2-0.4im    0.81+0.38im   0.81-0.38im
 0.72-0.67im  0.72+0.67im    0.1+0.12im    0.1-0.12im
 0.26+0.33im  0.26-0.33im  -0.56+0.71im  -0.56-0.71im

In [241]:
C = C_matrix(W)
matdisplay(C)

4×4 Matrix{ComplexF64}:
  -0.2-0.4im     0.6+0.53im  -0.26-0.33im  0.72-0.67im
  -0.2+0.4im     0.6-0.53im  -0.26+0.33im  0.72+0.67im
 -0.81-0.38im  -0.14-0.58im   0.56-0.71im   0.1+0.12im
 -0.81+0.38im  -0.14+0.58im   0.56+0.71im   0.1-0.12im

In [242]:
C = C_matrix_tullio(W)
matdisplay(C)

4×4 Matrix{ComplexF64}:
  -0.2-0.4im     0.6+0.53im  -0.26-0.33im  0.72-0.67im
  -0.2+0.4im     0.6-0.53im  -0.26+0.33im  0.72+0.67im
 -0.81-0.38im  -0.14-0.58im   0.56-0.71im   0.1+0.12im
 -0.81+0.38im  -0.14+0.58im   0.56+0.71im   0.1-0.12im

In [243]:
function C_tilde_matrix(W)
    return transpose(W)
end

C_tilde_matrix (generic function with 1 method)

In [244]:
function C_tilde_matrix_tullio(W)
    n = size(W, 1) ÷ 2
    W = reshape(W, 2, n, 2, n)
    @tullio C_tilde[β,j,δ,d] := W[δ,d,β,j]
    return reshape(C_tilde, 2n, 2n)
end

C_tilde_matrix_tullio (generic function with 1 method)

In [245]:
C = C_tilde_matrix(W)
matdisplay(C)

4×4 Matrix{ComplexF64}:
   0.6+0.53im   0.2+0.4im   0.72-0.67im   0.26+0.33im
   0.6-0.53im   0.2-0.4im   0.72+0.67im   0.26-0.33im
 -0.14-0.58im  0.81+0.38im   0.1+0.12im  -0.56+0.71im
 -0.14+0.58im  0.81-0.38im   0.1-0.12im  -0.56-0.71im

In [246]:
C = C_tilde_matrix_tullio(W)
matdisplay(C)

4×4 Matrix{ComplexF64}:
   0.6+0.53im   0.2+0.4im   0.72-0.67im   0.26+0.33im
   0.6-0.53im   0.2-0.4im   0.72+0.67im   0.26-0.33im
 -0.14-0.58im  0.81+0.38im   0.1+0.12im  -0.56+0.71im
 -0.14+0.58im  0.81-0.38im   0.1-0.12im  -0.56-0.71im

In [247]:
function B_tilde_matrix(A_inv)
    n = size(A_inv, 1) ÷ 2
    B_tilde = zero(A_inv)
    for g in 1:2:2n, j in 1:2:2n
        if j < g
            B_tilde[j,g] = -A_inv[j,g] + 1im * A_inv[j,g+1]
            B_tilde[j+1,g] = 1im * A_inv[j+1,g] - A_inv[j+1,g+1]
            B_tilde[j,g+1] = 1im * A_inv[j,g] + A_inv[j,g+1]
            B_tilde[j+1,g+1] = -A_inv[j+1,g] - 1im * A_inv[j+1,g+1]
        elseif j > g
            B_tilde[j,g] = A_inv[j,g] - 1im * A_inv[j,g+1]
            B_tilde[j+1,g] = -1im * A_inv[j+1,g] + A_inv[j+1,g+1]
            B_tilde[j,g+1] = -1im * A_inv[j,g] - A_inv[j,g+1]
            B_tilde[j+1,g+1] = A_inv[j+1,g] + 1im * A_inv[j+1,g+1]
        else # j == g #  
            B_tilde[j,g] = -A_inv[j,g+1]
            B_tilde[j+1,g] = A_inv[j+1,g]
            B_tilde[j,g+1] = -1im * A_inv[j,g+1]
            B_tilde[j+1,g+1] = -1im * A_inv[j+1,g]
        end
    end
    return B_tilde
end

B_tilde_matrix (generic function with 1 method)

In [248]:
function B_tilde_matrix_tullio(A_inv)
    n = size(A_inv, 1) ÷ 2
    # A_inv = reshape(A_inv, 2, n, 2, n)
    δ = Matrix(I(2))
    Δ = Matrix(I(n))
    @tullio D[γ,g,α,i] := Δ[g,i] * (δ[γ,1] + 1im * (-1)^α * δ[γ,2]) (α in 1:2)
    # matdisplay(reshape(D, 2n, 2n))
    D = reshape(D, 2n, 2n)
    # i = iα, j = jβ, g = gγ
    B = zero(A_inv)
    for g in 1:2n, j in 1:2n
        for i in 1:j-1
            B[j,g] += A_inv[j,i] * D[g,i]
        end
        for i in j+1:2n
            B[j,g] -= A_inv[j,i] * D[g,i]
        end
    end
    return B
end

B_tilde_matrix_tullio (generic function with 1 method)

In [249]:
A_inv |> matdisplay

4×4 Matrix{ComplexF64}:
 -0.0-0.0im    -1.0+0.0im   -1.87-0.19im  -0.0-0.0im
  1.0-0.0im     0.0-0.0im     0.0-0.0im   1.87-0.19im
 1.87+0.19im   -0.0+0.0im     0.0-0.0im   -1.0+0.0im
  0.0+0.0im   -1.87+0.19im    1.0-0.0im    0.0+0.0im

In [250]:
B_tilde = B_tilde_matrix(A_inv)
matdisplay(B_tilde)

4×4 Matrix{ComplexF64}:
   1.0-0.0im     0.0+1.0im    1.87+0.19im   0.19-1.87im
   1.0-0.0im    -0.0-1.0im   -1.87+0.19im  -0.19-1.87im
  1.87+0.19im   0.19-1.87im    1.0-0.0im     0.0+1.0im
 -1.87+0.19im  -0.19-1.87im    1.0-0.0im    -0.0-1.0im

In [251]:
B_tilde = B_tilde_matrix_tullio(A_inv)
matdisplay(B_tilde)

4×4 Matrix{ComplexF64}:
   1.0-0.0im     0.0+1.0im    1.87+0.19im   0.19-1.87im
   1.0-0.0im    -0.0-1.0im   -1.87+0.19im  -0.19-1.87im
  1.87+0.19im   0.19-1.87im    1.0-0.0im     0.0+1.0im
 -1.87+0.19im  -0.19-1.87im    1.0-0.0im    -0.0-1.0im

In [275]:
function fidelity_gradient(R)
    n = size(R, 1) ÷ 2
    W = W_matrix(R)
    
    # Todo: Reuse W here!
    A = fidelity_observable_matrix(R)
    val = pfaffian(A)
    A_inv = inv(A)
    
    B = B_matrix(A_inv)
    C = C_matrix(W)
    B_tilde = B_tilde_matrix(A_inv)
    C_tilde = C_tilde_matrix(W)
    
    # The real part of this is correct. But where is the imaginary part from?
    #             purely real             not purely real, a bug?
    return val * (1im * transpose(C) * B + transpose(C_tilde) * B_tilde) / 4
end

fidelity_gradient (generic function with 2 methods)

In [253]:
function fidelity_manual(R)
    A = fidelity_observable_matrix(MajoranaReg(R))
    return real(pfaffian(A))
end

fidelity_manual (generic function with 1 method)

In [254]:
function fidelity_gradient_fdm(R)
    grad(central_fdm(5, 1), fidelity_manual, R)
end

fidelity_gradient_fdm (generic function with 1 method)

In [272]:
R = reg1.state

6×6 Matrix{Float64}:
 -0.408238    0.626119    0.110955   -0.209045   0.019824   -0.620414
 -0.433502    0.165828    0.117592    0.821059  -0.246684    0.189098
  0.562909    0.212708    0.779704    0.126915  -0.0998383  -0.0622469
  0.56018     0.456798   -0.584169    0.339005   0.077611   -0.123823
 -0.0715667  -0.0990622   0.149642    0.263141   0.941411   -0.0847027
 -0.0981186   0.562809    0.0477093  -0.286159   0.191072    0.743604

In [276]:
fidelity_gradient(reg1.state) .|> real |> matdisplay

6×6 Matrix{Float64}:
 -0.19   0.44   0.25  -0.16   0.05  -0.24
 -0.39  -0.03   0.13   0.45  -0.01   0.1
  0.36  -0.0    0.43   0.18  -0.08   0.05
  0.22   0.33  -0.31   0.31  -0.04  -0.09
  0.06  -0.08   0.03   0.04   0.58  -0.06
  0.03   0.24   0.05  -0.1    0.09   0.52

In [274]:
fidelity_gradient_fdm(reg1.state)[1] |> matdisplay

6×6 Matrix{Float64}:
 -0.19   0.44   0.25  -0.16   0.05  -0.24
 -0.39  -0.03   0.13   0.45  -0.01   0.1
  0.36  -0.0    0.43   0.18  -0.08   0.05
  0.22   0.33  -0.31   0.31  -0.04  -0.09
  0.06  -0.08   0.03   0.04   0.58  -0.06
  0.03   0.24   0.05  -0.1    0.09   0.52

In [357]:
function wrapped_pfaffian(R)
    R = R - transpose(R)
    R = SkewHermitian(R)
    return pfaffian(R)
end

wrapped_pfaffian (generic function with 1 method)

In [358]:
A = rand(4, 4)
A = A - transpose(A)
# A = SkewHermitian(A)
grad(central_fdm(5, 1), wrapped_pfaffian, A)[1]

4×4 Matrix{Float64}:
  2.84086e-15   1.19022      -0.17811       1.68581
 -1.19022       2.84086e-15   0.249298     -0.412555
  0.17811      -0.249298      2.84086e-15   0.772645
 -1.68581       0.412555     -0.772645      2.84086e-15

In [360]:
-2pfaffian(A) * inv(A)

4×4 Matrix{Float64}:
  9.35812e-17   1.19022      -0.17811       1.68581
 -1.19022      -1.75749e-17   0.249298     -0.412555
  0.17811      -0.249298      1.75749e-17   0.772645
 -1.68581       0.412555     -0.772645     -0.0

## Debugging 1 qubit with Symbolics

In [45]:
@variables φ

1-element Vector{Num}:
 φ

In [59]:
@variables c, s

2-element Vector{Num}:
 c
 s

In [66]:
R = [cos(φ) -sin(φ); sin(φ) cos(φ)]

2×2 Matrix{Num}:
 cos(φ)  -sin(φ)
 sin(φ)   cos(φ)

In [60]:
R = [c -s; s c]

2×2 Matrix{Num}:
 c  -s
 s   c

In [130]:
R = Symbolics.variables(:R, 1:2, 1:2)

2×2 Matrix{Num}:
 R₁ˏ₁  R₁ˏ₂
 R₂ˏ₁  R₂ˏ₂

In [212]:
function fidelity_gradient(R::Matrix{Num})
    n = size(R, 1) ÷ 2
    W = W_matrix(R)
    
    # Todo: Reuse W here!
    A = fidelity_observable_matrix(R)
    # display(A)
    val = A[1,2] # pfaffian(A) # only consider 1 qubit case here
    A_inv = inv(A)
    # display(A)
    # display(A_inv)
    
    B = B_matrix(A_inv)
    C = C_matrix(W)
    B_tilde = B_tilde_matrix(A_inv)
    C_tilde = C_tilde_matrix(W)
    
    return val * (1im * transpose(C) * B + transpose(C_tilde) * B_tilde)
end

fidelity_gradient (generic function with 2 methods)

In [182]:
fg = fidelity_gradient(R)

2×2 Matrix{Complex{Num}}:
 (R₁ˏ₁ / ((1//4)*(R₁ˏ₁^2 + R₁ˏ₂^2 + R₂ˏ₁^2 + R₂ˏ₂^2) + (1//4)*(2R₁ˏ₁*R₂ˏ₂ - 2R₁ˏ₂*R₂ˏ₁)) + R₂ˏ₂ / ((1//4)*(R₁ˏ₁^2 + R₁ˏ₂^2 + R₂ˏ₁^2 + R₂ˏ₂^2) + (1//4)*(2R₁ˏ₁*R₂ˏ₂ - 2R₁ˏ₂*R₂ˏ₁)) + (-R₂ˏ₂) / (-(1//4)*(R₁ˏ₁^2 + R₁ˏ₂^2 + R₂ˏ₁^2 + R₂ˏ₂^2) - (1//4)*(2R₁ˏ₁*R₂ˏ₂ - 2R₁ˏ₂*R₂ˏ₁)) - R₁ˏ₁*(1 / (-(1//4)*(R₁ˏ₁^2 + R₁ˏ₂^2 + R₂ˏ₁^2 + R₂ˏ₂^2) - (1//4)*(2R₁ˏ₁*R₂ˏ₂ - 2R₁ˏ₂*R₂ˏ₁))))*((1//4)*(R₁ˏ₁^2 + R₁ˏ₂^2 + R₂ˏ₁^2 + R₂ˏ₂^2) + (1//4)*(2R₁ˏ₁*R₂ˏ₂ - 2R₁ˏ₂*R₂ˏ₁)) + ((1//4)*(R₁ˏ₁^2 + R₁ˏ₂^2 + R₂ˏ₁^2 + R₂ˏ₂^2) + (1//4)*(2R₁ˏ₁*R₂ˏ₂ - 2R₁ˏ₂*R₂ˏ₁))*((-R₂ˏ₁) / (-(1//4)*(R₁ˏ₁^2 + R₁ˏ₂^2 + R₂ˏ₁^2 + R₂ˏ₂^2) - (1//4)*(2R₁ˏ₁*R₂ˏ₂ - 2R₁ˏ₂*R₂ˏ₁)) + R₁ˏ₂ / ((1//4)*(R₁ˏ₁^2 + R₁ˏ₂^2 + R₂ˏ₁^2 + R₂ˏ₂^2) + (1//4)*(2R₁ˏ₁*R₂ˏ₂ - 2R₁ˏ₂*R₂ˏ₁)) + (-R₂ˏ₁) / ((1//4)*(R₁ˏ₁^2 + R₁ˏ₂^2 + R₂ˏ₁^2 + R₂ˏ₂^2) + (1//4)*(2R₁ˏ₁*R₂ˏ₂ - 2R₁ˏ₂*R₂ˏ₁)) + R₁ˏ₂*(1 / (-(1//4)*(R₁ˏ₁^2 + R₁ˏ₂^2 + R₂ˏ₁^2 + R₂ˏ₂^2) - (1//4)*(2R₁ˏ₁*R₂ˏ₂ - 2R₁ˏ₂*R₂ˏ₁))))*im  …  ((1//4)*(R₁ˏ₁^2 + R₁ˏ₂^2 + R₂ˏ₁^2 + R₂ˏ₂^2) + (1//4)*(2R₁ˏ₁*R₂ˏ₂ - 2R₁ˏ

In [183]:
@variables N
@variables D

1-element Vector{Num}:
 D

In [185]:
for _ in 1:10
    fg = substitute.(fg, Ref(Dict(R[1,1]^2 + R[1,2]^2 + R[2,1]^2 + R[2,2]^2 => N,)))
    fg = substitute.(fg, Ref(Dict(2R[1,1] * R[2,2] - 2R[2,1] * R[1,2] => 2D,)))
end
simplify.(fg)
# # Symbolics.Symbolics.expand.(fg)

2×2 Matrix{Complex{Num}}:
  (64//1)*((1//32)*R₁ˏ₁ + (1//32)*R₂ˏ₂)  …  (64//1)*((1//32)*R₁ˏ₂ - (1//32)*R₂ˏ₁)
 (64//1)*(-(1//32)*R₁ˏ₂ + (1//32)*R₂ˏ₁)     (64//1)*((1//32)*R₁ˏ₁ + (1//32)*R₂ˏ₂)

In [161]:
f = fidelity_observable_matrix(R)
for _ in 1:10
    fg = substitute.(f, Ref(Dict(R[1,1]^2 + 2R[1,1] * R[2,2] + R[1,2]^2 - 2R[1,2]*R[2,1] + R[2,1]^2 + R[2,2]^2 => N + 2D,)))
end
simplify.(f)

2×2 Matrix{Complex{Num}}:
                                                             0          …  (1//4)*(R₁ˏ₁^2 + 2R₁ˏ₁*R₂ˏ₂ + R₁ˏ₂^2 - 2R₁ˏ₂*R₂ˏ₁ + R₂ˏ₁^2 + R₂ˏ₂^2)
 (-1//4)*(R₁ˏ₁^2 + 2R₁ˏ₁*R₂ˏ₂ + R₁ˏ₂^2 - 2R₁ˏ₂*R₂ˏ₁ + R₂ˏ₁^2 + R₂ˏ₂^2)                                                                0

In [166]:
val = f[1,2]
# for _ in 1:10
#     val = substitute(val, Dict(R[1,1]^2 + R[1,2]^2 + R[2,1]^2 + R[2,2]^2 => N,))
#     val = substitute(val, Dict(2R[1,1] * R[2,2] - 2R[2,1] * R[1,2] => 2D,))
# end
# val

(1//4)*(R₁ˏ₁^2 + R₁ˏ₂^2 + R₂ˏ₁^2 + R₂ˏ₂^2) + (1//4)*(2R₁ˏ₁*R₂ˏ₂ - 2R₁ˏ₂*R₂ˏ₁)

In [158]:
4*f[1,2] |> simplify

R₁ˏ₁^2 + 2R₁ˏ₁*R₂ˏ₂ + R₁ˏ₂^2 - 2R₁ˏ₂*R₂ˏ₁ + R₂ˏ₁^2 + R₂ˏ₂^2

## Leftovers

In [7]:
reg1 = FLOYao.rand_state(3)
reg2 = FLOYao.zero_state(3)

MajoranaReg{Float64} with 3 qubits:
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0

In [8]:
fidelity(reg1, reg2)

0.9482898284113278

In [9]:
A = fidelity_observable_matrix(reg1)
# W = simplify.(W; expand=true)
# A = simplify.(A; expand=true)
# display(W)
display(A)

norm(A_asym + transpose(A_asym)) = 7.665777581559047e-17


6×6 Matrix{ComplexF64}:
  0.0+0.0im   1.0+0.0im   0.0-0.0im  -0.0+0.0im   0.0+0.0im   0.0+0.0im
  1.0+0.0im   0.0-0.0im  -0.0-0.0im   0.0+0.0im   0.0-0.0im   0.0-0.0im
  0.0-0.0im  -0.0-0.0im   0.0+0.0im   1.0+0.0im   0.0+0.0im  -0.0+0.0im
 -0.0+0.0im   0.0+0.0im   1.0+0.0im   0.0-0.0im  -0.0-0.0im   0.0-0.0im
  0.0+0.0im   0.0-0.0im   0.0+0.0im  -0.0-0.0im  -0.0+0.0im   1.0+0.0im
  0.0+0.0im   0.0-0.0im  -0.0+0.0im   0.0-0.0im   1.0+0.0im  -0.0-0.0im

6×6 Matrix{ComplexF64}:
        0.0+0.0im          …   0.0253894-0.0184753im
  -0.941825-4.90866e-18im       0.171419-0.0458542im
   0.128725+0.0652438im       -0.0278649-0.00587318im
  0.0258619-0.028667im         0.0890688-0.174211im
   0.171419+0.0458542im         0.922414+5.64611e-19im
 -0.0253894+0.0184753im    …         0.0+0.0im

In [10]:
sqrt(abs(pfaffian(A)))

0.9482898284113277